In [5]:
from ptflops import get_model_complexity_info
import torchvision.models as models

model = models.resnet50()

flops, params = get_model_complexity_info(model, (3, 224, 224), as_strings=True, print_per_layer_stat=False)

print(f"FLOPs: {flops}, Params: {params}")

FLOPs: 4.13 GMac, Params: 25.56 M


In [13]:
import torch
from torchinfo import summary
import torchvision.models as models

model = models.resnet50()

# flops, params = get_model_complexity_info(model, (3, 224, 224), as_strings=True, print_per_layer_stat=False)

input_tensor = (3, 224, 224)

summary(model, input_tensor)

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [Conv2d: 1]

In [6]:
!pip install torchinfo

In [154]:
import torch
from torchinfo import summary
import torchvision.models as models

model = models.resnet18()

# flops, params = get_model_complexity_info(model, (3, 224, 224), as_strings=True, print_per_layer_stat=False)

input_tensor = [1, 3, 224, 224]

summary(model, input_tensor)

ModuleNotFoundError: No module named 'torchinfo'

**summary(model, input_tensor, col_names=["input_size", "output_size", "num_params", "mult_adds"])**

In [19]:
model = models.resnet18()

# flops, params = get_model_complexity_info(model, (3, 224, 224), as_strings=True, print_per_layer_stat=False)

input_tensor = [1, 3, 224, 224]

summary(model, input_tensor, col_names=["input_size", "output_size", "num_params", "mult_adds"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Mult-Adds
ResNet                                   [1, 3, 224, 224]          [1, 1000]                 --                        --
├─Conv2d: 1-1                            [1, 3, 224, 224]          [1, 64, 112, 112]         9,408                     118,013,952
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         [1, 64, 112, 112]         128                       128
├─ReLU: 1-3                              [1, 64, 112, 112]         [1, 64, 112, 112]         --                        --
├─MaxPool2d: 1-4                         [1, 64, 112, 112]         [1, 64, 56, 56]           --                        --
├─Sequential: 1-5                        [1, 64, 56, 56]           [1, 64, 56, 56]           --                        --
│    └─BasicBlock: 2-1                   [1, 64, 56, 56]           [1, 64, 56, 56]           --                        --
│    │ 

**FOR CPU INFERENCE TIME**

In [46]:

import torch
import torch.nn as nn
from torch.autograd import profiler

# Example model
model = nn.Linear(10, 1)

# Dummy input
input_tensor = torch.randn(1, 10)

# Measure inference time
with profiler.profile(record_shapes=True) as prof:
    with profiler.record_function("model_inference"):
        output = model(input_tensor)

print(prof.key_averages().table(sort_by="cpu_time_total"))


----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
       model_inference        53.55%     156.549us       100.00%     292.330us     292.330us             1  
          aten::linear         3.82%      11.166us        46.45%     135.781us     135.781us             1  
           aten::addmm        17.48%      51.089us        24.76%      72.392us      72.392us             1  
               aten::t        10.40%      30.397us        17.86%      52.223us      52.223us             1  
       aten::transpose         5.74%      16.782us         7.47%      21.826us      21.826us             1  
           aten::copy_         4.26%      12.455us         4.26%      12.455us      12.455us             1  
          aten::exp

**FOR GPU INFERENCE TIME**

In [58]:

import torch
import torch.nn as nn
from torch.autograd import profiler

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



# Example model
model = nn.Linear(10, 1).to(device)

# Dummy input
input_tensor = torch.randn(1, 10).to(device)



# Measure inference time
with profiler.profile(record_shapes=True) as prof:
    with profiler.record_function("model_inference"):
        output = model(input_tensor)

print(prof.key_averages().table(sort_by="cuda_time_total"))


--------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
--------------------  ------------  ------------  ------------  ------------  ------------  ------------  
     model_inference        12.60%     145.459us       100.00%       1.154ms       1.154ms             1  
        aten::linear         1.21%      13.982us        87.40%       1.009ms       1.009ms             1  
             aten::t         2.51%      28.947us         4.31%      49.746us      49.746us             1  
     aten::transpose         1.41%      16.326us         1.80%      20.799us      20.799us             1  
    aten::as_strided         0.67%       7.769us         0.67%       7.769us       3.885us             2  
         aten::addmm        77.02%     888.970us        81.88%     945.062us     945.062us             1  
        aten::expand         0.79%   

In [76]:
import torch
import torch.nn as nn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Example model
model = nn.Linear(10, 1).to(device)

# Dummy input
input_tensor = torch.randn(1, 10).to(device)

# Measure inference time
start_event = torch.cuda.Event(enable_timing=True)
end_event = torch.cuda.Event(enable_timing=True)

start_event.record()
output = model(input_tensor)
end_event.record()

# Waits for everything to finish running
torch.cuda.synchronize()

inference_time = start_event.elapsed_time(end_event) / 1000  # Convert to seconds
print(f"Inference time: {inference_time:.6f} seconds")


Inference time: 0.000396 seconds


In [113]:
import torch
import torch.nn as nn
import time

device = "cpu" # torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Example model
model = nn.Linear(10, 1).to(device)

# Dummy input
input_tensor = torch.randn(1, 10).to(device)

# Measure inference time
start_time = time.time()
output = model(input_tensor)
end_time = time.time()

inference_time = end_time - start_time
print(f"Inference time: {inference_time:.6f} seconds")


Inference time: 0.000275 seconds


In [153]:

import torch
import torch.nn as nn
import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = nn.Linear(10, 1).to(device)   # Example model
input_tensor = torch.randn(1, 10).to(device)   # Dummy input


torch.cuda.synchronize()     # Ensure any previous GPU operations are complete
start_time = time.time()    # Measure inference time

output = model(input_tensor)   # Perform the inference

torch.cuda.synchronize()     # Ensure the inference is complete
end_time = time.time()


inference_time = end_time - start_time


print(f"Inference time on GPU: {inference_time:.6f} seconds")


Inference time on GPU: 0.000474 seconds
